Prep

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
  
# Read the CSV file.
data = pd.read_csv('data.csv', skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

Step 2. Training the baseline classifier

We can now start to investigate different classifiers.

The DummyClassifier
Links to an external site. is a simple classifier that does not make use of the features: it just returns the most common label in the training set, in this case Spondylolisthesis. The purpose of using such a stupid classifier is as a baseline: a simple classifier that we can try before we move on to more complex classifiers.

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='most_frequent')

To get an idea of how well our simple classifier works, we carry out a cross-validation Links to an external site. over the training set and compute the classification accuracy on each fold.

In [6]:
from sklearn.model_selection import cross_val_score

cross_val_score(clf, Xtrain, Ytrain)

NameError: name 'clf' is not defined

Step 3. Trying out some different classifiers

Replace the DummyClassifier with some more meaningful classifier and run the cross-validation again. Try out a few classifiers and see how much you can improve the cross-validation accuracy. Remember, the accuracy is defined as the proportion of correctly classified instances, and we want this value to be high.

Here are some possible options:

Tree-based classifiers:

    sklearn.tree.DecisionTreeClassifier 

Links to an external site.
sklearn.ensemble.RandomForestClassifier
Links to an external site.
sklearn.ensemble.GradientBoostingClassifier

    Links to an external site.

Linear classifiers:

    sklearn.linear_model.Perceptron 

Links to an external site.
sklearn.linear_model.LogisticRegression
Links to an external site.
sklearn.svm.LinearSVC

    Links to an external site.

Neural network classifier (will take longer time to train):

    sklearn.neural_network.MLPClassifier 

    Links to an external site.

You may also try to tune the hyperparameters of the various classifiers to improve the performance. For instance, the decision tree classifier has a parameter that sets the maximum depth, and in the neural network classifier you can control the number of layers and the number of neurons in each layer.

-----------------------------------------------------

Step 4. Final evaluation

When you have found a classifier that gives a high accuracy in the cross-validation evaluation, train it on the whole training set and evaluate it on the held-out test set.

In [7]:
from sklearn.metrics import accuracy_score
  
clf.fit(Xtrain, Ytrain)
Yguess = clf.predict(Xtest)
print(accuracy_score(Ytest, Yguess))

NameError: name 'clf' is not defined

For the report. In your submitted report, include a list of three classifiers you tried in Step 3 and their accuracies, add a description of the classifier you selected in Step 4 and report its accuracy. (At this point, we are of course not asking you to describe internal workings of various machine learning models that we will cover in detail at later points during the course, but you are of course free to read about them if you're interested.)